<a href="https://colab.research.google.com/github/RafSar2020/Python-Project-for-Data-Science/blob/main/ejercicio_sentimientos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creando un modelo de clasificación de texto

Objetivos de la práctica:
- Conocer cómo crear un algoritmo de clasificación de texto usando la librería transformers.
- Conocer el entorno de HuggingFace ([datasets](https://huggingface.co/datasets), [models](https://huggingface.co/models), [spaces](https://huggingface.co/spaces),...)

Este notebook está basado en el [curso de HuggingFace](https://huggingface.co/course/chapter3/1?fw=pt).

Para este notebook es conveniente que compruebes que la opción de GPU está activada (Runtime -> Change Runtime Type).

## Creando una cuenta en HuggingFace

Lo primero que debemos hacer es [crear una cuenta de HuggingFace](https://huggingface.co/join). Además deberás crear un [token de escritura](https://huggingface.co/docs/hub/security-tokens). Estos dos pasos solo los deberás hacer la primera vez.

## Instalando librerías

Por defecto, el entorno de Google Colab no tiene instaladas las librerías de [HuggingFace](https://huggingface.co/), por lo que vamos a hacer en primer lugar es instalar las librerías: [Transformers](https://huggingface.co/docs/transformers/index), [Datasets](https://huggingface.co/docs/datasets/index), y [Evaluate](https://huggingface.co/docs/evaluate/index).

In [1]:
!pip install datasets evaluate transformers[sentencepiece] accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00


A continuación nos conectamos al hub de huggingface, lo que nos permitirá subir nuestros modelos a este entorno. Al ejecutar la siguiente celda aparecerá un widget en el cual tendremos que copiar el token generado en el primer paso y pulsar en el botón login.

In [1]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset

Para este ejemplo vamos a utilizar el [dataset de "dair-ai-emotion"](https://huggingface.co/datasets/dair-ai/emotion) que contiene emociones de mensajes de tweeter en inglés. Para cada tweeter se incluye una valoración entre 1 y 6, que se corresponden con las siguientes emociones: anger, fear, joy, love, sadness, y surprise. Nuestro objetivo es crear un modelo para automatizar la valoración de los sentimientos.

Comenzamos descargando el dataset.

In [2]:
from datasets import load_dataset
raw_dataset = load_dataset("emotion")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Veamos el contenido de este dataset.

In [3]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

Podemos ver que tenemos un objeto DatasetDict que puede verse como un diccionario. Dicho diccionario contiene un atributo `train`. En algunos casos veremos que el dataset ya está divido en conjuntos de entrenamiento y test, pero en este caso no es así, por lo que lo tendremos que dividirlo nosotros. Pero antes de esto vamos a ver alguna de las frases del dataset, para lo que tenemos que acceder al atributo `train`.

In [4]:
raw_dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

Con el anterior comando vemos que tenemos un Dataset con dos columnas: `text`y `label`. Si queremos ver el contenido del dataset, lo podemos transformar a formato pandas y verlo como una tabla.  

In [6]:
raw_dataset['train'].to_pandas()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3


Para poder entrenar un modelo con este dataset es necesario tokenizarlo. Cada modelo tokeniza de una manera distinta, por lo que es necesario indicar el modelo para tokenizar el texto. En nuestro caso vamos a utilizar un modelo llamado [Electricidad](https://huggingface.co/mrm8488/electricidad-base-discriminator).

In [7]:
from transformers import AutoTokenizer, DataCollatorWithPadding

model_checkpoint = "mrm8488/electricidad-base-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

Definimos una función para tokenizar el texto. Notar que para otros datasets
será necesario cambiar el valor de "review_summary" por la columna que queramos
tokenizar, el resto del código no hará falta tocarlo.

In [8]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

Tokenizamos el dataset y lo mostramos.

In [9]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

Podemos ver que han aparecido tres nuevas columnas ('input_ids', 'token_type_ids' y 'attention_mask') que serán utilizadas para entrenar el modelo.

Para poder entrenar un modelo de clasificación de texto, es necesario que nuestro dataset tenga una columna llamada `label`, por lo que tenemos que renombrar nuestra columna `star_rating`.

In [17]:
tokenized_dataset = tokenized_dataset.rename_column('star_rating','label')
tokenized_dataset

ValueError: Original column name star_rating not in the dataset. Current columns in the dataset: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask']

Además, necesitamos partir nuestro dataset en un conjunto de entrenamiento y en un conjunto de test. Para lo cual, vamos a:
1. Revolver el dataset.
2. Calcular el número de elementos de nuestro dataset.
3. Dividir el dataset en dos trozos (80% para entrenar y 20% para testear).
4. Construir un nuevo dataset con un conjunto de entrenamiento y uno de test.

Notar que este paso es necesario porque el dataset no está dividido previamente en entrenamiento y test, si ese fuera el caso, este paso no sería necesario.

In [26]:
from datasets import DatasetDict,Dataset
# 1. Revolvemos el dataset con el método shuffle
new_tokenized_dataset = tokenized_dataset["train"].shuffle()
# 2. Calculamos el número de elementos del dataset
len_dataset = len(tokenized_dataset["train"])
# 3. Partimos el dataset en dos trozos
train_dataset = tokenized_dataset["train"][0:int(len_dataset*0.8)]
test_dataset = tokenized_dataset["train"][int(len_dataset*0.8):]



Por último, antes de definir nuestro modelo tenemos que definir una función que se va a encargar de preparar los datos para que sean procesados de manera eficiente por el modelo.

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Modelo

Pasamos ahora a definir el modelo, lo primero que vamos a definir son los argumentos con los que vamos a entrenar nuestro modelo. Aunque podemos configurar el entrenamiento de múltiples maneras, en este caso vamos a utilizar los valores por defecto, y solo vamos a modificar el nombre con el que se va a guardar nuestro modelo, que en este caso va a ser `clasificador-sentimientos`. Además le vamos a pedir que nos muestre cómo de bien funciona el modelo a medida que se va entrenando mediante la `evaluation_strategy` con valor `epoch`.

In [11]:
from transformers import TrainingArguments
training_args = TrainingArguments("clasificador-sentimientos",evaluation_strategy="epoch")

A continuación definimos nuestro modelo, para ello usamos la clase `AutoModelForSequenceClassification` y vamos a utilizar un modelo pre-entrenado (recordar lo que era el transfer learning). Para ello solo tenemos que indicar el nombre de nuestro modelo (definido previamente en la variable `model_checkpoint` y el número de posibles valores que puede tomar nuestro clasificador (en este caso 5).  

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=6)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at mrm8488/electricidad-base-discriminator and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ahora definimos la función que usaremos para calcular la precisión de nuestro modelo. En este caso usaremos la accuracy.

In [14]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

Ya podemos definir nuestro objeto `trainer` que usaremos para entrenar nuestro modelo. La estructura de este objeto será siempre la misma. Le tenemos que proporcionar:
1. El modelo.
2. La configuración del entrenamiento.
3. El conjunto de entrenamiento.
4. El conjunto de test.
5. El objeto que prepara los datos.
6. El tokenizador.
7. La métrica.

In [33]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Directamente el conjunto de entrenamiento
    eval_dataset=test_dataset,    # Directamente el conjunto de prueba
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)




In [35]:
for i in range(len(train_dataset)):
    try:
        _ = train_dataset[i]
    except KeyError:
        print(f"KeyError encountered at index: {i}")


KeyError encountered at index: 0
KeyError encountered at index: 1
KeyError encountered at index: 2
KeyError encountered at index: 3
KeyError encountered at index: 4


Se ajusta el código al hecho de que el conjunto de datos ya estaba dividido en train/test data

Y ahora entrenamos el modelo mediante el método `train`. Este proceso puede llevar unos minutos y entrenará el modelo por 3 épocas (es decir mostrará todos los datos al modelo 3 veces). Este valor se puede cambiar en [la configuración del entrenamiento](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments).  

In [34]:
trainer.train()

KeyError: 3

Hemos obtenido una accuracy de aproximadamente el 44%. Esto puede variar de ejecución en ejecución ya que el entrenamiento de los modelos siempre tiene un factor aleatorio.

## Compartiendo el modelo

Una vez que tenemos entrenado nuestro modelo, nos interesa compartirlo con el resto del mundo para que puedan usarlo y también compararlo con otros modelos.

Es por ello que vamos a subir nuestro modelo al hub de huggingface. Para ello tenemos que ejecutar el siguiente comando.

In [ ]:
# Vamos a la carpeta donde se ha guardado nuestro modelo, es el valor que
# definimos previamente en el objeto TrainingArguments
%cd clasificador-muchocine
# Subimos el modelo indicando un mensaje de confirmación, y una etiqueta.
trainer.push_to_hub(commit_message="Training complete", tags="classification")

/content/clasificador-muchocine


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RafSarLop/clasificador-muchocine/commit/36b10ddf8111bc5555d8e2530f3d7358c6499e8d', commit_message='Training complete', commit_description='', oid='36b10ddf8111bc5555d8e2530f3d7358c6499e8d', pr_url=None, pr_revision=None, pr_num=None)

Al terminar de ejecutarse el comando anterior tendremos nuestro modelo disponible en https://huggingface.co/RafSarLop/clasificador-muchocine (en la URL anterior deberás reemplazar joheras por tu nombre de usuario).

Si accedes al enlace anterior, verás que tienes tu modelo disponible y una [tarjeta de modelo (o *model card*)](https://huggingface.co/docs/hub/model-cards) con una breve descripción del mismo. Es conveniente que proporciones información adicional a la *model card* ya que la que se genera de forma automática es demasiado básica.

Además verás que en el enlace anterior tienes un pequeño widget que te permite hacer predicciones con tu modelo.

Finalmente, vamos a ver cómo usar nuestro modelo para hacer predicciones desde código (esto puede ser útil sí por ejemplo nos interesa procesar múltiples textos de manera secuencial).


## Usando el modelo

En este caso al ser un modelo que hemos entrenado nosotros mismos podríamos usar los ficheros locales, pero vamos a ver cómo usar el modelo que acabamos de subir al hub de HuggingFace.

Para ello usamos un `pipeline` al que le debemos indicar el nombre del modelo que queremos descargar.

In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model='joheras/clasificador-muchocine')

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/730k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Ahora podemos hacer predicciones con nuestro modelo que tomará valores de label_0 (1 estrella) a label_4 (5 estrellas).

In [ ]:
classifier('Es una obra mestra. Brillante.')

[{'label': 'LABEL_3', 'score': 0.4971005618572235}]

In [ ]:
classifier('Es una película muy buena.')

[{'label': 'LABEL_3', 'score': 0.7302730679512024}]

In [ ]:
classifier('Una buena película, sin más.')

[{'label': 'LABEL_3', 'score': 0.7171180248260498}]

In [ ]:
classifier('Esperaba mucho más.')

[{'label': 'LABEL_2', 'score': 0.7051416635513306}]

In [ ]:
classifier('He tirado el dinero. Una basura. Vergonzoso.')

[{'label': 'LABEL_0', 'score': 0.5983389019966125}]

Como podemos ver con los ejemplos anteriores, a pesar de que la accuracy del modelo no era excesivamente alta, para las frases anteriores funciona casi siempre correctamente. Con esto hemos visto cómo entrenar un modelo, compartirlo con el mundo, y usarlo.